In [ ]:
%pip install sacremoses
%pip install -U bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "ModelSpace/GemmaX2-28-2B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left')

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", load_in_8bit=True)

In [ ]:
import json
with open('arabic_stance_master.json', 'r') as fin:
  data = json.load(fin)

In [ ]:
import re

prog = re.compile(r'MENTION\s*')
for k, v in data.items():
  ar = v[0]
  ar_clean = prog.sub('', ar)
  data[k] = [v[0], ar_clean, v[-1]]

In [ ]:
data['0'][1]

In [ ]:
len(data)

In [ ]:
outdata = []

In [ ]:
import gc

# for_translation = ['بس التطعيمات الي اولادنا اخذوها دي اخذت وقتها في تصنيعها وتجارب الي ان طلعت لناس لكن تطعيم كورونا ماخلص من التجارب لسه وبيطبق علينا بس حرام الاطفال كمان تدخل في نظام التجارب دي واحنا لسه مو عارفين مضاعفاته وابعاده ايش هي بعدين',
# "#خذ_الخطوه_بلغ_عن_اضرار_اللقاح #لا_للتطعيم_الاجباري186 #لا_للتطعيم_الاجباري #متضرري_اللقاح #ولي_العهد كورونا #الجرعة_التنشيطية #الجرعة_الثالثة تكفى ياملكنا وقف اللقاحات الإجبارية منشنو ان شاء الله يشوف تغريداتنا"
# ]

template = "Translate this from Arabic to English:\nArabic: {}\nEnglish:"

for i in range(0, 1200, 100):
  input_sents = [template.format(x[1]) for x in list(data.values())[i:i+100]]
  inputs = tokenizer(input_sents, return_tensors="pt", padding=True)
  inputs.to('cuda')
  outputs = model.generate(**inputs, max_new_tokens=150)
  out = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  outdata.extend(out)
  del inputs
  gc.collect()

In [ ]:
import csv

with open('ar_to_en.csv', 'w') as fout:
  csv.writer(fout).writerows([[x.split('\n')[-1].split(':', maxsplit=1)[1]] for x in outdata])

In [ ]:
new_json = {k: v[:-1] + [outdata[i].split('\n')[-1].split(':', maxsplit=1)[1]] + [v[-1]] for i, (k, v) in enumerate(data.items())}

In [ ]:
with open('arabic_stance_trans_master.json', 'w') as fout:
  json.dump(new_json, fout, ensure_ascii=False, indent=4)

# Перевод на русский

In [ ]:
import json

with open('arabic_stance_trans_master.json', 'r') as fin:
  data = json.load(fin)

In [ ]:
outdata = []

In [ ]:
import gc
from tqdm.notebook import tqdm

template = "Translate this from English to Russian:\nEnglish: {}\nRussian:"

for i in tqdm(range(0, 1200, 100)):
  input_sents = [template.format(x[-2]) for x in list(data.values())[i:i+100]]
  inputs = tokenizer(input_sents, return_tensors="pt", padding=True)
  inputs.to('cuda')
  outputs = model.generate(**inputs, max_new_tokens=150)
  out = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  outdata.extend(out)
  del outputs, out
  del inputs
  gc.collect()

In [ ]:
outdata[1]

In [ ]:
def get_translation(s):
  return s.split('\n')[-1].split(':', maxsplit=1)[1]

In [ ]:
new_json = {k: v[:-1] + [get_translation(outdata[i])] + [v[-1]] for i, (k, v) in enumerate(data.items())}

In [ ]:
with open('arabic_stance_trans_grandmaster.json', 'w') as fout:
  json.dump(new_json, fout, ensure_ascii=False, indent=4)